In [7]:
import pandas as pd
from goodreads import client

In [5]:
books = pd.read_csv('good_books_10k/good_books_long.csv')

In [8]:
goodreads_key = 't6mIkabukH29jAey0381yA'
goodreads_secret = '6IWvvO5CNFIAqUragee2Bb5HkEOvxIYqSXeXdFSHvM'
good_client = client.GoodreadsClient(goodreads_key, goodreads_secret)

In [0]:
# get descriptions for books in the dataset using the book_ids
import time
good_descriptions = []
i = 1
for book_id in books['goodreads_book_id']:
    try:
        print(i)
        i += 1
        book = good_client.book(book_id)
        good_descriptions.append(book.description)
        time.sleep(0.5)
    except:
        pass

In [0]:
books['description'] = pd.Series(good_descriptions)

In [9]:
books.head()

,goodreads_book_id,authors,original_publication_year,original_title,average_rating,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,description
0,2767052,Suzanne Collins,2008.0,The Hunger Games,4.34,66715,127936,560092,1481305,2706317,"Could you survive on your own, in the wild, wi..."
1,3,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,4.44,75504,101676,455024,1156318,3011543,Harry Potter's life is miserable. His parents ...
2,41865,Stephenie Meyer,2005.0,Twilight,3.57,456191,436802,793319,875073,1355439,<b>About three things I was absolutely positiv...
3,2657,Harper Lee,1960.0,To Kill a Mockingbird,4.25,60427,117415,446835,1001952,1714267,The unforgettable novel of a childhood in a sl...
4,4671,F. Scott Fitzgerald,1925.0,The Great Gatsby,3.89,86236,197621,606158,936012,947718,Alternate Cover Edition ISBN: 0743273567 (ISBN...


In [9]:
# function to remove the 'Alternate Cover Edition ...' paragraph at the beginning 
def clean_cover_isbn(text):
  try:
    text = text.split('<br />')[2]
    return text
  except:
    return text

In [10]:
# function to remove html markings
from bs4 import BeautifulSoup
def clean_html(text):
  try:
    soup = BeautifulSoup(text, 'lxml')
    clean_text = soup.get_text()
    return clean_text
  except:
    return text

In [11]:
# function to remove punctuation
import string
def clean_punctuations(text):
  clean_text = "".join([c for c in text if c not in string.punctuation])
  return clean_text

In [0]:
# somehow there were books that were skipped during the first scrape, so here we go again
retrieve_books = books[books['description'].isnull()]
retrieve_books.reset_index(drop = True, inplace = True)

In [39]:
retrieve_books.shape

(125, 11)

In [0]:
import time
retrieve_good_descriptions = []
i = 1
for book_id in retrieve_books['goodreads_book_id']:
    try:
        print(i)
        i += 1
        book = good_client.book(book_id)
        retrieve_good_descriptions.append(book.description)
        time.sleep(0.5)
    except:
        pass

In [0]:
retrieve_good_descriptions

In [0]:
retrieve_books['description'] = pd.Series(retrieve_good_descriptions)

In [0]:
retrieve_books = retrieve_books[~retrieve_books['description'].isnull()]

In [62]:
books.head()

,goodreads_book_id,authors,original_publication_year,original_title,average_rating,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,description
0,2767052,Suzanne Collins,2008.0,The Hunger Games,4.34,66715,127936,560092,1481305,2706317,"Could you survive on your own, in the wild, wi..."
1,3,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,4.44,75504,101676,455024,1156318,3011543,Harry Potter's life is miserable. His parents ...
2,41865,Stephenie Meyer,2005.0,Twilight,3.57,456191,436802,793319,875073,1355439,<b>About three things I was absolutely positiv...
3,2657,Harper Lee,1960.0,To Kill a Mockingbird,4.25,60427,117415,446835,1001952,1714267,The unforgettable novel of a childhood in a sl...
4,4671,F. Scott Fitzgerald,1925.0,The Great Gatsby,3.89,86236,197621,606158,936012,947718,Alternate Cover Edition ISBN: 0743273567 (ISBN...


In [0]:
info_books = books[~books['description'].isnull()]

In [0]:
info_books = pd.concat([info_books, retrieve_books])

In [75]:
info_books.shape

(9979, 11)

In [0]:
info_books['description'] = info_books['description'].apply(lambda x: clean_cover_isbn(x))
info_books['description'] = info_books['description'].apply(lambda x: clean_html(x))
info_books['description'] = info_books['description'].apply(lambda x: clean_punctuations(x))

In [77]:
info_books.head()

,goodreads_book_id,authors,original_publication_year,original_title,average_rating,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,description
0,2767052,Suzanne Collins,2008.0,The Hunger Games,4.34,66715,127936,560092,1481305,2706317,In the ruins of a place once known as North Am...
1,3,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,4.44,75504,101676,455024,1156318,3011543,After a lifetime of bottling up his magical po...
2,41865,Stephenie Meyer,2005.0,Twilight,3.57,456191,436802,793319,875073,1355439,First Edward was a vampire
3,2657,Harper Lee,1960.0,To Kill a Mockingbird,4.25,60427,117415,446835,1001952,1714267,Compassionate dramatic and deeply moving To Ki...
4,4671,F. Scott Fitzgerald,1925.0,The Great Gatsby,3.89,86236,197621,606158,936012,947718,The Great Gatsby F Scott Fitzgeralds third boo...


In [78]:
from keras.preprocessing.text import text_to_word_sequence
def get_tokens(text):
  return text_to_word_sequence(text)

Using TensorFlow backend.


In [0]:
info_books['description'] = info_books['description'].apply(lambda x: get_tokens(x))

In [44]:
info_books = info_books[info_books['description'].map(lambda x: len(x) > 2)]

In [ ]:
info_books.to_csv('good_books_10k/good_books_tokenized.csv', index = False)

In [13]:
def get_title(book_id):
    try:
        book = good_client.book(book_id)
        return book.title
    except:
        print(book_id)
        pass

In [14]:
info_books['original_title'] = info_books['goodreads_book_id'].apply(lambda x: get_title(x))

12033455
14201
1067
33600
34084
29800
23754
21686
341879
567616
6596
12985
5960325
402093
10585
22543496
16148398
25690958
27406436
30364931


In [0]:
# unsupervised first
# cluster first then assign label from labelled data points to entire cluster
# train supervised models